In [1]:
import copy

import numpy as np
from scipy.special import binom

from games import ParameterizedSparseLinearModel
from approximators import SHAPIQEstimator, PermutationSampling
from approximators.regression import RegressionEstimator

### Setup the game
Here we use the linear model as we can compute the ground truth interaction values directly from the game.

In [2]:
# setup the game function (here we use a
game = ParameterizedSparseLinearModel(
    n=30, # n of players
    weighting_scheme="uniform", # how the interactions should be distributed over the subset sizes
    min_interaction_size=1, # min size of interactions in the model
    max_interaction_size=20, # max size of interactions in the model
    n_interactions=100, # number of interactions in the model
    n_non_important_features=3 # number of dummy (zero weight) features, which will also not be part of the interactions
)

game_name = game.game_name
game_fun = game.set_call
n = game.n
N = set(range(n))

### Set up the interaction index
The interaction values will be calculated for the order of interactions.

In [3]:
interaction_order = 2

### Setup the baseline approximators
In total there are two approaches for the interaction indices. Permutation sampling is defined for SII and STI. The weighted least-squares approach is defined for SFI.

In [4]:
# Permutation Sampling for Shapley Interaction Index
shapley_extractor_sii_permutation = PermutationSampling(
    N=N,
    max_order=interaction_order,
    min_order=interaction_order,
    interaction_type="SII"
)

# Permutation Sampling for Shapley Taylor Index
shapley_extractor_sti_permutation = PermutationSampling(
    N=N,
    max_order=interaction_order,
    min_order=interaction_order,
    interaction_type="STI")

# Regression Estimator for Shapley Faith Index
shapley_extractor_sfi_regression = RegressionEstimator(
    N=N,
    max_order=interaction_order
)

baselines = {
    "SII": shapley_extractor_sii_permutation,
    "STI": shapley_extractor_sti_permutation,
    "SFI": shapley_extractor_sfi_regression
}

### Setup SHAP-IQ approximator
SHAP-IQ is defined for all interaction indices that follow a general definition of Shapley interaction (SII, STI, and SFI). For more information we refer to the full paper (the section about SI).

In [5]:
# SHAP-IQ to approximate the Shapley Interaction Index
shapley_extractor_sii = SHAPIQEstimator(
    N=N,
    max_order=interaction_order,
    min_order=interaction_order,
    interaction_type="SII"
)

# SHAP-IQ to approximate the Shapley Taylor Index
shapley_extractor_sti = SHAPIQEstimator(
    N=N,
    max_order=interaction_order,
    min_order=interaction_order,
    interaction_type="STI"
)

# SHAP-IQ to approximate the Shapley Faith Index
shapley_extractor_sfi = SHAPIQEstimator(
    N=N,
    max_order=interaction_order,
    min_order=interaction_order,
    interaction_type="SFI"
)

approximators = {
    "SII": shapley_extractor_sii,
    "STI": shapley_extractor_sti,
    "SFI": shapley_extractor_sfi
}

### Run the approximators
Run the baseline and SHAP-IQ approximators. Also compute the ground truth interaction values.

In [6]:
budget = 2**14

baseline_results = {}
shap_iq_results = {}
ground_truth_results = {}
for interaction_type in {'SII', 'STI', 'SFI'}:
    baseline = baselines[interaction_type]
    shap_iq = approximators[interaction_type]

    # run baseline method
    approx_value = baseline.approximate_with_budget(
        game_fun, budget
    )
    baseline_results[interaction_type] = copy.deepcopy(approx_value)

    # run shap_iq method
    approx_value = shap_iq.compute_interactions_from_budget(
        game=game.set_call,
        budget=budget,
        pairing=False,
        sampling_kernel="ksh",
        sampling_only=False,
        stratification=False
    )
    shap_iq_results[interaction_type] = copy.deepcopy(approx_value)

    # get ground truths (only possible this way with the sparse linear model, otherwise we need to use brute force)
    ground_truth_results[interaction_type] = copy.deepcopy(
        game.exact_values(
            gamma_matrix=shap_iq.weights,
            min_order=interaction_order,
            max_order=interaction_order
        )
    )

Exact values: pre-computed weights: 100%|██████████| 60/60 [00:00<00:00, 7510.17it/s]
Exact values: Final computation: 100%|██████████| 43500/43500.0 [00:00<00:00, 864788.55it/s]
Exact values: pre-computed weights: 100%|██████████| 60/60 [00:00<00:00, 7492.95it/s]
Exact values: Final computation: 100%|██████████| 43500/43500.0 [00:00<00:00, 845578.78it/s]
Exact values: pre-computed weights: 100%|██████████| 60/60 [00:00<00:00, 7376.76it/s]
Exact values: Final computation: 100%|██████████| 43500/43500.0 [00:00<00:00, 860169.27it/s]


In [7]:
def mse(gt, approx):
    return np.sum((approx - gt) ** 2) / binom(n, interaction_order)

def mae(gt, approx):
    return np.sum(abs(approx - gt)) / binom(n, interaction_order)

### Results for SII

In [23]:
ground_truth = ground_truth_results['SII'][interaction_order]
ground_truth[0:10,0:10]

array([[0.        , 0.59045346, 0.61679992, 0.66305406, 1.03892973,
        1.09497735, 0.80572489, 0.96682754, 0.70174181, 0.5918839 ],
       [0.        , 0.        , 0.66203722, 0.96368035, 0.93264247,
        0.76499057, 0.75009169, 0.82467037, 0.76702708, 0.6000354 ],
       [0.        , 0.        , 0.        , 0.61314871, 0.65491188,
        0.75705393, 0.58338959, 0.80600865, 0.58566827, 0.81739913],
       [0.        , 0.        , 0.        , 0.        , 0.79349788,
        0.86915699, 0.76363011, 0.72511788, 0.87368551, 0.73394485],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.12237228, 0.96228723, 1.24501365, 1.02441111, 0.85157799],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.65363361, 0.99548605, 0.89701147, 0.6039882 ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.86306115, 0.77996014, 0.73736438],
       [0.        , 0.        , 0.       

In [24]:
shap_iq_approx = shap_iq_results['SII'][interaction_order]
shap_iq_approx[0:10,0:10]

array([[0.        , 0.7020692 , 0.419596  , 0.57169201, 0.48565707,
        1.69212508, 0.46424538, 1.08989029, 0.66370394, 0.59824909],
       [0.        , 0.        , 0.70216958, 1.16275952, 0.82915714,
        0.63962151, 0.71942777, 0.66476537, 1.11389939, 0.38796942],
       [0.        , 0.        , 0.        , 0.66307322, 0.83320578,
        0.44035951, 0.4630146 , 0.81797068, 0.18491171, 0.72984137],
       [0.        , 0.        , 0.        , 0.        , 1.06893909,
        0.69219831, 0.60942459, 0.77649392, 0.5310322 , 0.60191847],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.2708638 , 1.34288271, 1.30220284, 0.94733497, 0.73289649],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.8070091 , 0.9987037 , 0.86662587, 0.46161378],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.48984502, 0.72782042, 0.87657824],
       [0.        , 0.        , 0.       

In [26]:
baseline_approx = baseline_results['SII']
baseline_approx[0:10,0:10]

array([[0.        , 0.7459507 , 0.67554694, 1.25024553, 2.89645089,
        0.34084647, 1.59663349, 0.        , 0.52979954, 1.28390438],
       [0.        , 0.        , 1.26533311, 0.23100447, 1.34776655,
        1.68891918, 0.02511217, 0.2133376 , 0.86598356, 0.06058482],
       [0.        , 0.        , 0.        , 1.25016498, 0.00696915,
        0.58795679, 0.        , 0.16108491, 1.31025644, 2.64351182],
       [0.        , 0.        , 0.        , 0.        , 0.33536439,
        1.77698978, 0.64570379, 0.59815733, 0.56291902, 0.11778886],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        1.3998797 , 2.72511628, 0.73212402, 1.83163683, 2.70198155],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.28330418, 2.45149523, 0.32577192, 0.3349095 ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.02354462, 1.48909419, 0.14803406],
       [0.        , 0.        , 0.       

In [11]:
print("average MSE (SHAP-IQ):", mse(ground_truth, shap_iq_approx))
print("average MSE (baseline):", mse(ground_truth, baseline_approx))
print("average MAE (SHAP-IQ):", mae(ground_truth, shap_iq_approx))
print("average MAE (baseline-IQ):", mae(ground_truth, baseline_approx))

average MSE (SHAP-IQ): 0.05491215164573437
average MSE (baseline): 0.3426715523866486
average MAE (SHAP-IQ): 0.181830796153039
average MAE (baseline-IQ): 0.43374917745220604


### Results for STI
The tables show only a 10:10 selection as it's easier to render on the browser.

In [20]:
ground_truth = ground_truth_results['STI'][interaction_order]
ground_truth[0:10,0:10]

array([[0.        , 0.08521511, 0.09246183, 0.09515917, 0.18197315,
        0.26589965, 0.13610853, 0.14911822, 0.10868903, 0.10017396],
       [0.        , 0.        , 0.12403223, 0.19556555, 0.19333123,
        0.16595596, 0.10018417, 0.16167617, 0.1090114 , 0.08381847],
       [0.        , 0.        , 0.        , 0.09231001, 0.09358691,
        0.1177208 , 0.08772728, 0.13078073, 0.07613014, 0.19111465],
       [0.        , 0.        , 0.        , 0.        , 0.11696789,
        0.13223659, 0.1079567 , 0.10484025, 0.12353606, 0.11012413],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.23261584, 0.16228535, 0.2366328 , 0.16478696, 0.13932503],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.106401  , 0.20862171, 0.15480466, 0.09914012],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.12057757, 0.10585839, 0.11011258],
       [0.        , 0.        , 0.       

In [21]:
shap_iq_approx = shap_iq_results['STI'][interaction_order]
shap_iq_approx[0:10,0:10]

array([[ 0.00000000e+00,  8.81573893e-02,  3.13022762e-02,
         1.30708665e-01,  1.91404947e-01,  3.84415197e-01,
         1.34769704e-01,  3.11463909e-01, -9.67780307e-03,
        -7.95258939e-02],
       [ 0.00000000e+00,  0.00000000e+00,  8.14512098e-02,
         1.58296326e-01,  1.23392498e-01,  9.13348059e-02,
        -1.69673757e-02,  2.32296034e-01,  1.71981648e-01,
         4.64339439e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.52491399e-01,  7.40759411e-02,  1.87847518e-01,
         1.03877102e-01,  4.77228503e-02,  2.09883709e-01,
         2.20317170e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  2.16363924e-01,  1.38000532e-01,
         6.27807701e-02,  7.47717165e-02,  1.09858253e-01,
         1.37077747e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.66143493e-01,
         3.03608637e-01,  2.64482675e-01,  1.59722776e-01,
         2.

In [27]:
baseline_approx = baseline_results['STI']
baseline_approx[0:10,0:10]

array([[0.        , 0.        , 0.4241554 , 0.33566995, 0.59582069,
        0.        , 0.        , 0.06537901, 0.42942259, 0.04001014],
       [0.        , 0.        , 0.25631952, 0.09499119, 1.53480158,
        0.        , 0.37749589, 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.11855072, 0.25075615,
        0.        , 0.05411627, 0.        , 0.10213339, 0.04246746],
       [0.        , 0.        , 0.        , 0.        , 0.37117809,
        0.        , 0.13913114, 0.12265927, 0.36448572, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.0868169 , 0.25100501, 0.41248539, 1.87280077, 0.08262101],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.25818601, 0.        , 0.        ],
       [0.        , 0.        , 0.       

In [15]:
print("average MSE (SHAP-IQ):", mse(ground_truth, shap_iq_approx))
print("average MSE (baseline):", mse(ground_truth, baseline_approx))
print("average MAE (SHAP-IQ):", mae(ground_truth, shap_iq_approx))
print("average MAE (baseline-IQ):", mae(ground_truth, baseline_approx))

average MSE (SHAP-IQ): 0.005934886685855929
average MSE (baseline): 0.04072448198834724
average MAE (SHAP-IQ): 0.0609165457879775
average MAE (baseline-IQ): 0.10898474109872716


### Results for SFI

In [16]:
ground_truth = ground_truth_results['SFI'][interaction_order]
ground_truth[0:10,0:10]

array([[0.        , 0.21889443, 0.23684334, 0.24530352, 0.44926322,
        0.58131037, 0.33752985, 0.3778691 , 0.27456531, 0.24954168],
       [0.        , 0.        , 0.29365853, 0.4534531 , 0.4427552 ,
        0.38001889, 0.26039754, 0.37376559, 0.28155678, 0.21741233],
       [0.        , 0.        , 0.        , 0.23505921, 0.24090936,
        0.29915021, 0.22032956, 0.32699285, 0.19975792, 0.40754664],
       [0.        , 0.        , 0.        , 0.        , 0.29956726,
        0.33784114, 0.27798427, 0.26932122, 0.31974372, 0.28238614],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.54315755, 0.40303973, 0.56107428, 0.41533443, 0.35063285],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.26632984, 0.48380388, 0.38229509, 0.24988164],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.3114523 , 0.27539923, 0.27886257],
       [0.        , 0.        , 0.       

In [17]:
shap_iq_approx = shap_iq_results['SFI'][interaction_order]
shap_iq_approx[0:10,0:10]

array([[ 0.        ,  0.40393996,  0.44288946,  0.25154228,  0.58793872,
         0.57766946,  0.49371314,  0.35225549,  0.2504753 , -0.05174908],
       [ 0.        ,  0.        , -0.04039069,  0.70287247,  0.6858006 ,
         0.5416274 ,  0.54022561,  0.32081382,  0.18444517,  0.27838112],
       [ 0.        ,  0.        ,  0.        ,  0.48345237,  0.16995588,
         0.36741649,  0.32937478,  0.59094345,  0.14183357,  0.43872295],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.23276257,
         0.47590402,  0.44803548,  0.13498862,  0.22438219,  0.43206497],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.7186783 ,  0.25302747,  0.46300209,  0.64379333,  0.36757757],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.15351125,  0.29579483,  0.16806373,  0.22899422],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.37757568

In [28]:
baseline_approx = baseline_results['SFI']
baseline_approx[0:10,0:10]

array([[ 0.        ,  0.25776843,  0.32882847,  0.26074043,  1.15507934,
         1.26369485,  0.73659646,  1.12589965,  0.0891565 ,  0.01353147],
       [ 0.        ,  0.        ,  0.52201055,  1.16100115,  0.89671465,
         0.59814011,  0.88062103,  0.37114863,  0.94742762,  0.44229431],
       [ 0.        ,  0.        ,  0.        ,  0.42939902,  0.26848008,
         1.04501161,  0.35002032,  0.66391884,  0.29652033,  0.85603322],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.55965544,
         1.01607061,  0.66595564,  0.22440603,  1.0744013 ,  0.83652158],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.12469592,  1.00211745,  1.62856877,  1.0362138 ,  0.7667705 ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        , -0.00685555,  0.67019505,  1.01090945,  0.04272738],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.57173781

In [19]:
print("average MSE (SHAP-IQ):", mse(ground_truth, shap_iq_approx))
print("average MSE (baseline):", mse(ground_truth, baseline_approx))
print("average MAE (SHAP-IQ):", mae(ground_truth, shap_iq_approx))
print("average MAE (baseline-IQ):", mae(ground_truth, baseline_approx))

average MSE (SHAP-IQ): 0.026582299828491064
average MSE (baseline): 0.19889758064600255
average MAE (SHAP-IQ): 0.13197026641569165
average MAE (baseline-IQ): 0.3314616906813085
